# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re
import typing

from random import randint
from functools import wraps

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'
BOOK_PATH2 = 'https://www.gutenberg.org/files/5827/5827-8.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter() 
        result = func(*args, **kwargs)
        tm = time.perf_counter() - start
        print(f"Время выполнения функции {func.__name__}: {tm:.17f}")
        return result

    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"Функция вызвана с параметрами: {args}, {kwargs}")
        return result

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        # кол-во вызовов храним в объекте декоратора
        wrapper.count += 1
        print(f"Функция была вызывана {wrapper.count} раз")
        return result

    # задаем начальное значение счетчика в декораторе
    wrapper.count = 0
    return wrapper

## Задание 4

In [6]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func,
    чьи аргументы args должны быть хешируемыми
    """

    @wraps(func)
    def fmemo(*args):
        # проверяем, что переданные аргументы хэшируемые
        if all(isinstance(arg, typing.Hashable) for arg in args):
            # если да, то их можно использовать как ключ в словаре fmemo.cache
            if args in fmemo.cache:
                # если аргумент уже в кэше, возвращаем сохраненное значение
                result = fmemo.cache[args]
            else:
                # иначе сохраняем результат ф-ии и возвращаем его
                result = fmemo.cache[args] = func(*args)
            return result
        else:
            # если аргументы не хэшируемые, просто вызываем ф-ю
            return func(*args)

    fmemo.cache = {}
    return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

In [8]:
print(word_count('whole'))

Время выполнения функции word_count: 1.78787540000121226
Функция вызвана с параметрами: ('whole',), {}
Функция была вызывана 1 раз
Cлово whole встречается 176 раз


In [9]:
print(word_count('knowledge', url=BOOK_PATH2))

Время выполнения функции word_count: 1.12936499999705120
Функция вызвана с параметрами: ('knowledge',), {'url': 'https://www.gutenberg.org/files/5827/5827-8.txt'}
Функция была вызывана 2 раз
Cлово knowledge встречается 308 раз


In [10]:
# вариант без кэширования
# используется вложенная ф-я _fib, чтобы замерять время выполнения в целом, а не каждого рекурсивного вызова
@benchmark
def fib(n):
    def _fib(n):
        if n < 2:
            return n
        return _fib(n-2) + _fib(n-1)
    
    return _fib(n)

# измеряем время выполнения
result1 = fib(30)

Время выполнения функции fib: 0.09628700000030221


In [11]:
# вариант с кэшированием
# такая же вложенная ф-я, но результаты промежуточных рекурсивных вызовов будут запомнены
@benchmark
def fib_cached(n):
    @memo
    def _fib(n):
        if n < 2:
            return n
        return _fib(n-2) + _fib(n-1)
    
    return _fib(n)

# измеряем время выполнения с кэшированием
# время выполнения должно значительно уменьшиться
result2 = fib_cached(30)

Время выполнения функции fib_cached: 0.00009559999671183


In [12]:
# проверяем, что результаты сходятся
assert result1 == result2